In [1]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

from contentBasedFilteringModel import item_ids, user_profiles,tfidf_matrix
from init import articles_df, model_evaluator


# users: 1895
# users with at least 5 interactions: 1140
# of interactions: 72312
# of interactions from users with at least 5 interactions: 69868
# of unique user/news item interactions: 39106
# interactions on Train set: 31284
# interactions on Test set: 7822
********Content-Based Filtering Model*********
(1, 5000)


In [2]:

class ContentBasedRecommender:

    MODEL_NAME = 'Content-Based'

    def __init__(self, items_df=None):
        self.item_ids = item_ids
        self.items_df = items_df

    def get_model_name(self):
        return self.MODEL_NAME

    def _get_similar_items_to_user_profile(self, person_id, topn=1000):
        #Computes the cosine similarity between the user profile and all item profiles
        cosine_similarities = cosine_similarity(user_profiles[person_id], tfidf_matrix)
        #Gets the top similar items
        similar_indices = cosine_similarities.argsort().flatten()[-topn:]
        #Sort the similar items by similarity
        similar_items = sorted([(item_ids[i], cosine_similarities[0,i]) for i in similar_indices], key=lambda x: -x[1])
        return similar_items

    def recommend_items(self, user_id, items_to_ignore=[], topn=10, verbose=False):
        similar_items = self._get_similar_items_to_user_profile(user_id)
        #Ignores items the user has already interacted
        similar_items_filtered = list(filter(lambda x: x[0] not in items_to_ignore, similar_items))

        recommendations_df = pd.DataFrame(similar_items_filtered, columns=['contentId', 'recStrength']) \
                                    .head(topn)

        if verbose:
            if self.items_df is None:
                raise Exception('"items_df" is required in verbose mode')

            recommendations_df = recommendations_df.merge(self.items_df, how = 'left',
                                                          left_on = 'contentId',
                                                          right_on = 'contentId')[['recStrength', 'contentId', 'title', 'url', 'lang']]


        return recommendations_df

content_based_recommender_model = ContentBasedRecommender(articles_df)

print('Evaluating Content-Based Filtering model...')
cb_global_metrics, cb_detailed_results_df = model_evaluator.evaluate_model(content_based_recommender_model)
print('\nGlobal metrics:\n%s' % cb_global_metrics)
cb_detailed_results_df.head(10)


Evaluating Content-Based Filtering model...
Running evaluation for users
1139 users processed

Global metrics:
{'modelName': 'Content-Based', 'recall@5': 0.41459984658655075, 'recall@10': 0.5241626182562005}


,hits@5_count,hits@10_count,interacted_count,recall@5,recall@10,_person_id
76,16,26,192,0.083333,0.135417,3609194402293569455
17,21,35,134,0.156716,0.261194,-2626634673110551643
16,22,34,130,0.169231,0.261538,-1032019229384696495
10,34,54,117,0.290598,0.461538,-1443636648652872475
82,8,15,88,0.090909,0.170455,-2979881261169775358
161,14,23,80,0.175000,0.287500,-3596626804281480007
65,10,15,73,0.136986,0.205479,1116121227607581999
81,11,20,69,0.159420,0.289855,692689608292948411
106,5,10,69,0.072464,0.144928,-9016528795238256703
52,4,11,68,0.058824,0.161765,3636910968448833585
